Create a clean schema for this project

In [0]:
display(spark.sql("SHOW SCHEMAS IN btris_dbx"))

spark.sql("CREATE SCHEMA IF NOT EXISTS btris_dbx.observability")
display(spark.sql("SHOW SCHEMAS IN btris_dbx"))

databaseName
default
information_schema
observability


databaseName
default
information_schema
observability


Create the Volume (this is where you’ll store the raw Excel/CSV)

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS btris_dbx.observability.server_observability_vol;
DESCRIBE VOLUME btris_dbx.observability.server_observability_vol;

name,catalog,database,owner,storage_location,volume_type,comment,securable_type,securable_kind
server_observability_vol,btris_dbx,observability,eaglesanalytica@gmail.com,abfss://unity-catalog-storage@dbstoragew5zdakkxxnami.dfs.core.windows.net/4480026081424261/__unitystorage/catalogs/291a63c4-3c5c-4da9-87d7-c3f7affce7f9/volumes/f433e967-9633-4fc8-bffd-8d734219ef42,MANAGED,null,VOLUME,VOLUME_STANDARD


Verify the physical Volume path

In [0]:
display(dbutils.fs.ls("/Volumes/btris_dbx/observability/server_observability_vol"))

[]

Create clean folder structure inside the Volume

In [0]:
base_path = "/Volumes/btris_dbx/observability/server_observability_vol"

dbutils.fs.mkdirs(f"{base_path}/raw/perfmon")
dbutils.fs.mkdirs(f"{base_path}/raw/windows_events")

display(dbutils.fs.ls(base_path))

path,name,size,modificationTime
dbfs:/Volumes/btris_dbx/observability/server_observability_vol/raw/,raw/,0,1771363224000


Verify file readability

In [0]:
base_path = "/Volumes/btris_dbx/observability/server_observability_vol/raw"

display(dbutils.fs.ls(f"{base_path}/perfmon"))
display(dbutils.fs.ls(f"{base_path}/windows_events"))

path,name,size,modificationTime
dbfs:/Volumes/btris_dbx/observability/server_observability_vol/raw/perfmon/perfmon_metrics_2025-04-29.xls,perfmon_metrics_2025-04-29.xls,133120,1771363516000


path,name,size,modificationTime
dbfs:/Volumes/btris_dbx/observability/server_observability_vol/raw/windows_events/AllServerEvents_2026-02-08.csv,AllServerEvents_2026-02-08.csv,3820003,1771363551000


Test reading the Windows Events CSV

In [0]:
events_path = "/Volumes/btris_dbx/observability/server_observability_vol/raw/windows_events/AllServerEvents_2026-02-08.csv"

events_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(events_path)
)

display(events_df.limit(5))

Message,ID,ProviderName,LogName,MachineName,TimeCreated,ContainerLog,LevelDisplayName,CreatedDate
The processing of Group Policy failed. Windows could not obtain the name of a domain controller. This could be caused by a name resolution failure. Verify your Domain Name System (DNS) is configured and working correctly.,1054,Microsoft-Windows-GroupPolicy,System,HC1DBSQ27PV,2025-11-14T06:32:28.000Z,System,Error,2025-11-14T09:37:48.447Z
Microsoft Windows Server has detected that NTLM authentication is presently being used between clients and this server. This event occurs once per boot of the server on the first time a client uses NTLM with this server. NTLM is a weaker authentication mechanism. Please check: Which applications are using NTLM authentication? Are there configuration issues preventing the use of stronger authentication such as Kerberos authentication? If NTLM must be supported is Extended Protection configured? Details on how to complete these checks can be found at http://go.microsoft.com/fwlink/?LinkId=225699.,6038,LsaSrv,System,HC1DBSQ64PV,2025-11-14T00:50:34.000Z,System,Warning,2025-11-14T09:37:48.453Z
A timeout (30000 milliseconds) was reached while waiting for a transaction response from the 1EClient service.,7011,Service Control Manager,System,HC1DBSQ53PV,2025-11-14T04:12:30.000Z,System,Error,2025-11-14T09:37:48.453Z
The server {338B40F9-9D68-4B53-A793-6B9AA0C5F63B} did not register with DCOM within the required timeout.,10016,Microsoft-Windows-DistributedCOM,System,HC1DBSQ21PV,2025-11-14T08:48:16.000Z,System,Error,2025-11-14T09:37:48.457Z
A provider WPJProvider has been registered in the Windows Management Instrumentation namespace root\CCM\CIModels to use the LocalSystem account. This account is privileged and the provider may cause a security violation if it does not correctly impersonate user requests.,63,Microsoft-Windows-WMI,Application,AWSDBSQ62PV,2025-11-14T12:43:29.000Z,Application,Warning,2025-11-14T09:37:48.457Z


Create the Windows Events Delta table

In [0]:
from pyspark.sql import functions as F

# 1) Read CSV (same as you did)
events_path = "/Volumes/btris_dbx/observability/server_observability_vol/raw/windows_events/AllServerEvents_2026-02-08.csv"

events_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(events_path)
)

# 2) Normalize key + timestamps
events_clean = (
    events_df
    .withColumnRenamed("MachineName", "server_name")
    .withColumn("time_created", F.to_timestamp(F.col("TimeCreated").cast("string")))
    .withColumn("created_date", F.to_timestamp(F.col("CreatedDate").cast("string")))
)

# 3) Write Delta table
spark.sql("CREATE SCHEMA IF NOT EXISTS btris_dbx.observability")

(events_clean
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("btris_dbx.observability.windows_events_delta"))

# 4) Quick verify
display(spark.sql("SELECT server_name, time_created, LevelDisplayName, ProviderName, ID FROM btris_dbx.observability.windows_events_delta LIMIT 10"))

server_name,time_created,LevelDisplayName,ProviderName,ID
HC1DBSQ27PV,2025-11-14T06:32:28.000Z,Error,Microsoft-Windows-GroupPolicy,1054
HC1DBSQ64PV,2025-11-14T00:50:34.000Z,Warning,LsaSrv,6038
HC1DBSQ53PV,2025-11-14T04:12:30.000Z,Error,Service Control Manager,7011
HC1DBSQ21PV,2025-11-14T08:48:16.000Z,Error,Microsoft-Windows-DistributedCOM,10016
AWSDBSQ62PV,2025-11-14T12:43:29.000Z,Warning,Microsoft-Windows-WMI,63
AWSDBSQ62PV,2025-11-14T20:49:07.000Z,Warning,ESENT,640
HC1DBSQ53PV,2025-11-14T06:32:55.000Z,Error,Microsoft-Windows-GroupPolicy,1030
AWSDBSQ51PV,2025-11-14T22:06:50.000Z,Error,MSSQLSERVER,18059
HC1DBSQ45PV,2025-11-14T03:15:29.000Z,Error,EventLog,6008
HC1DBSQ36Pv,2025-11-14T17:13:58.000Z,Error,Microsoft-Windows-Security-Kerberos,4


Create the Perfmon Delta table from the perfmon CSV

In [0]:
import re

perfmon_path = "/Volumes/btris_dbx/observability/server_observability_vol/raw/perfmon/perfmon_metrics_2025-04-29.csv"

perf_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(perfmon_path)
)

def clean_col(c: str) -> str:
    c = c.strip().lower()
    c = re.sub(r"[%/]", " ", c)           # replace a few common symbols with space
    c = re.sub(r"[()]", "", c)            # drop parentheses
    c = re.sub(r"[^a-z0-9]+", "_", c)     # everything non-alnum -> underscore
    c = re.sub(r"_+", "_", c).strip("_")  # collapse underscores
    return c

# apply renaming
renamed = perf_df
for old in perf_df.columns:
    renamed = renamed.withColumnRenamed(old, clean_col(old))

# (optional) ensure server_name exists (handles "server name" -> "server_name")
if "server_name" not in renamed.columns:
    # find any likely server column
    for c in renamed.columns:
        if c in ["server", "servername", "machine", "machine_name"]:
            renamed = renamed.withColumnRenamed(c, "server_name")
            break

# write delta
(renamed.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("btris_dbx.observability.perfmon_metrics_delta"))

# verify
display(spark.sql("SELECT * FROM btris_dbx.observability.perfmon_metrics_delta LIMIT 5"))
print("Created Delta table: btris_dbx.observability.perfmon_metrics_delta ✅")


datetime,server_name,applicationname,max_cpu_utilization,cpuutilizationdate,max_memory_utilization,memoryutilizationdate,wait_type,wait_timems,most_expensive_query_by_cpu,most_expensive_query_by_cpu_url,most_expensive_query_by_physical_reads,most_expensive_query_by_physical_reads_url,jobsurl,cach_hit_ratio
2025-04-29,HC1DBSQ21PV,"AD Sync,SCCM,VAMT",35,2025-04-29 00:01:23.807000000,100,2025-04-29 00:09:51.687000000,SOS_SCHEDULER_YIELD,90102,"SELECT TOP 1 @last_internal_table_name = internal_table_name,",null,null,null,null,null
@last_start_time = start_time,,null,null,null,null,null,null,null,null,null,null,null,null,null
@last_end_time = end_time,null,null,null,null,null,null,null,null,null,null,null,null,null,null
@last_rows_cleaned_up = rows_cleaned_up,null,null,null,null,null,null,null,null,null,null,null,null,null,null
@last_cleanup_version = cleanup_version,null,null,null,null,null,null,null,null,null,null,null,null,null,null


Created Delta table: btris_dbx.observability.perfmon_metrics_delta ✅


Verify server_name alignment between both tables

In [0]:
spark.sql("""
SELECT DISTINCT server_name
FROM btris_dbx.observability.windows_events_delta
ORDER BY server_name
""").show(20, False)

spark.sql("""
SELECT DISTINCT server_name
FROM btris_dbx.observability.perfmon_metrics_delta
ORDER BY server_name
""").show(20, False)

+------------+
|server_name |
+------------+
|AWSDBSQ37PV |
|AWSDBSQ42PV |
|AWSDBSQ48PV |
|AWSDBSQ50PV |
|AWSDBSQ51PV |
|AWSDBSQ62PV |
|DR1DBSQLF1PV|
|HC1DBSQ21PV |
|HC1DBSQ23PV |
|HC1DBSQ25PV |
|HC1DBSQ27PV |
|HC1DBSQ28PV |
|HC1DBSQ32PV |
|HC1DBSQ33PV |
|HC1DBSQ35PV |
|HC1DBSQ35PV1|
|HC1DBSQ36PV |
|HC1DBSQ36Pv |
|HC1DBSQ39PV |
|HC1DBSQ39PV1|
+------------+
only showing top 20 rows
+---------------------------------------------------------------------------------------+
|server_name                                                                            |
+---------------------------------------------------------------------------------------+
|NULL                                                                                   |
|                                                                                       |
|                                                                                       |
| '$.id') IS NOT NULL                                                      

confirm the actual column names in the perfmon Delta table.

In [0]:
spark.sql("""
DESCRIBE btris_dbx.observability.perfmon_metrics_delta
""").show(100, False)

+------------------------------------------+---------+-------+
|col_name                                  |data_type|comment|
+------------------------------------------+---------+-------+
|datetime                                  |string   |NULL   |
|server_name                               |string   |NULL   |
|applicationname                           |string   |NULL   |
|max_cpu_utilization                       |string   |NULL   |
|cpuutilizationdate                        |string   |NULL   |
|max_memory_utilization                    |string   |NULL   |
|memoryutilizationdate                     |string   |NULL   |
|wait_type                                 |string   |NULL   |
|wait_timems                               |int      |NULL   |
|most_expensive_query_by_cpu               |string   |NULL   |
|most_expensive_query_by_cpu_url           |string   |NULL   |
|most_expensive_query_by_physical_reads    |string   |NULL   |
|most_expensive_query_by_physical_reads_url|string   |N

verify overlap between both tables — not just distinct values separately, but intersection.

In [0]:
spark.sql("""
SELECT DISTINCT w.server_name
FROM btris_dbx.observability.windows_events_delta w
INNER JOIN btris_dbx.observability.perfmon_metrics_delta p
    ON w.server_name = p.server_name
ORDER BY w.server_name
""").show(50, False)

+---------------+
|server_name    |
+---------------+
|AWSDBSQ50PV    |
|AWSDBSQ51PV    |
|AWSDBSQ62PV    |
|DR1DBSQLF1PV   |
|HC1DBSQ21PV    |
|HC1DBSQ23PV    |
|HC1DBSQ25PV    |
|HC1DBSQ27PV    |
|HC1DBSQ33PV    |
|HC1DBSQ35PV    |
|HC1DBSQ36PV    |
|HC1DBSQ39PV    |
|HC1DBSQ45PV    |
|HC1DBSQ53PV    |
|HC1DBSQ54PV    |
|HC1DBSQ57PV    |
|HC1DBSQ60PV    |
|HC1DBSQ62PV    |
|HC1DBSQ64PV    |
|HC1DBSQ70PV    |
|HC1DBSQ90PV    |
|LD5AIDBSQ01PV  |
|LX1APDS01PV    |
|WEUPRSOLUNSDW02|
+---------------+



Create a “server dimension” view/table to power the Streamlit dropdown

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW btris_dbx.observability.v_servers AS
SELECT DISTINCT w.server_name
FROM btris_dbx.observability.windows_events_delta w
INNER JOIN btris_dbx.observability.perfmon_metrics_delta p
  ON w.server_name = p.server_name
""")

# verify
spark.sql("SELECT * FROM btris_dbx.observability.v_servers ORDER BY server_name").show(100, False)

+---------------+
|server_name    |
+---------------+
|AWSDBSQ50PV    |
|AWSDBSQ51PV    |
|AWSDBSQ62PV    |
|DR1DBSQLF1PV   |
|HC1DBSQ21PV    |
|HC1DBSQ23PV    |
|HC1DBSQ25PV    |
|HC1DBSQ27PV    |
|HC1DBSQ33PV    |
|HC1DBSQ35PV    |
|HC1DBSQ36PV    |
|HC1DBSQ39PV    |
|HC1DBSQ45PV    |
|HC1DBSQ53PV    |
|HC1DBSQ54PV    |
|HC1DBSQ57PV    |
|HC1DBSQ60PV    |
|HC1DBSQ62PV    |
|HC1DBSQ64PV    |
|HC1DBSQ70PV    |
|HC1DBSQ90PV    |
|LD5AIDBSQ01PV  |
|LX1APDS01PV    |
|WEUPRSOLUNSDW02|
+---------------+



In [0]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()

for wh in w.warehouses.list():
    print(wh.name, wh.id, wh.state)

Serverless Starter Warehouse 47bde9279fec4222 State.STOPPED


In [0]:
%sql
-- Allow catalog access
GRANT USE CATALOG ON CATALOG btris_dbx
TO `a70123f0-da9c-444a-bcfa-eed1fa16a9d4`;

-- Allow schema access
GRANT USE SCHEMA ON SCHEMA btris_dbx.observability
TO `a70123f0-da9c-444a-bcfa-eed1fa16a9d4`;

-- Allow reading objects
GRANT SELECT ON VIEW btris_dbx.observability.v_servers
TO `a70123f0-da9c-444a-bcfa-eed1fa16a9d4`;

GRANT SELECT ON TABLE btris_dbx.observability.perfmon_metrics_delta
TO `a70123f0-da9c-444a-bcfa-eed1fa16a9d4`;

GRANT SELECT ON TABLE btris_dbx.observability.windows_events_delta
TO `a70123f0-da9c-444a-bcfa-eed1fa16a9d4`;
